<a href="https://colab.research.google.com/github/kdpark0284/HM_titledescription_filtering/blob/testing1/FilterTitles_Ko.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KoNLPy Start Up With Google Colab

## Install Dependencies

In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy
pip install openpyxl

In [ ]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

### Install Mecab (Takes 5min+ usually)

In [ ]:
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

In [ ]:
cd Mecab-ko-for-Google-Colab

In [ ]:
!bash install_mecab-ko_on_colab_light_220429.sh

## (Test before using KoNLPy) -- Not Neccesary

In [ ]:
%%bash
python3 -c "import jpype; jpype.startJVM(convertStrings=True); print(jpype.isJVMStarted())"

In [ ]:
%%bash
pip3 install pytest
TEST_DIR=$(mktemp -d -t test-XXXXXXXXXX)
git clone https://github.com/konlpy/konlpy --depth=1 $TEST_DIR
python3 -m pytest -v $TEST_DIR

## Use KoNLPy Examples

In [ ]:
# import konlpy
# from konlpy.tag import Kkma, Komoran, Hannanum, Okt
# from konlpy.utils import pprint
# kkma = Kkma()
# komoran = Komoran()
# hannanum = Hannanum()
# okt = Okt()
# pprint(kkma.sentences(u'네, 안녕하세요. 반갑습니다.'))
# pprint(komoran.nouns(u'질문이나 건의사항은 깃헙 이슈 트래커에 남겨주세요.'))
# pprint(hannanum.pos(u'오류보고는 실행환경, 에러메세지와함께 설명을 최대한상세히!^^'))
# pprint(okt.phrases(u'구글은 멋져요. 깃허브도 멋져요. KoNLPy도 멋져요!'))

If the installation of Mecab was successful, the code below will run well.

In [ ]:
# import konlpy
# from konlpy.utils import pprint
# from konlpy.tag import Mecab
# from konlpy.utils import pprint

# mecab = Mecab()

# text = "삼성 갤럭시 탭 s8 울트라(6GB+1TB)스페이스블랙[자급제]"
# mecabnouns = mecab.nouns(text)
# print("Mecab Nouns:", mecabnouns)
# mecabmorphs = mecab.morphs(text)
# print("Mecab Morphs:", mecabmorphs)
# type(mecabmorphs)

# # mecab.tagset

# Extraction Process (Read files, Extract Relevant Words)

## File Upload, Load Dependencies

In [ ]:
import pandas as pd
import re
import openpyxl
from konlpy.tag import Mecab
from konlpy.utils import pprint
from google.colab import files

mecab = Mecab()
uploaded_file = files.upload ()

"""
Excel 파일 업로드를 해주세요.
Please upload an Excel file.

"""

df = pd.read_excel('Revised_Description_rev2.xlsx')

"""
Excel 파일 이름과 '일치'하게 해주세요.
Please enter the file name and 'match' it.
예시)
test.xlsx 를 업로드한다면 "df = pd.read_xlsx('test.xlsx')"
test.csv 를 업로드한다면 "df = pd.read_csv('test.csv')"
To upload test.xlsx, please enter "df = pd.read_xlsx('test.xlsx')"
To upload test.csv, please enter "df = pd.read_csv('test.csv')"
"""


## Filter Lists

In [ ]:
product_title = [
                '갤럭시탭', '갤럭시노트', '갤럭시버즈','아이폰', '갤럭시', '프로', '울트라', '맥스', '아이패드', '에어팟', '플립', '폴드', '뮤패드', 'FE', 'FE+'
                '북', '맥북', 'S', 'Ultra', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'Pro', 'ProMax', '+', 'A',
                'Galaxy', 'iPhone', 'iPad', 'MacBook', 'Zenbook', '그램', '에어', '스타일', '프로맥스', '미패드'
                  ]

product_type_dict = {
    "PC" : "IT",
    "모바일" : "IT",
    "태블릿" : "IT",
    "노트북" : "IT",
    "데스크탑" : "IT",
    "NAFS" : "IT",
    "백색" : "ETC",
    "A/V" : "ETC"
}

color_list = ["문스톤그레이", "스페이스 그레이", "코스믹 그레이", "내추럴", "카밍 차콜", "카밍그린", "오로라화이트", "잉크웰그레이",
              "스노우화이트", "새틴 화이트", "프라임 실버", "라벤더", "스페이스그레이", "루미다크실버", "미드나이트", "플래티넘실버",
              "스페이스 블랙", "그라파이트", "매트블랙", "사파이어", "스타라이트", "이클립스그레이", "새틴화이트", "보라퍼플",
              "글램화이트", "네이비블루", "메탈릭실버", "화이트실버", "카밍베이지", "페블 그레이", "블랙 메탈", "코타 화이트",
              "코타 퍼플", "크림라벤더", "크림 화이트", "크림화이트", "코타 차콜", "실버글라스", "블랙 크롬메탈", "샤인베이지",
              "에센셜화이트", "카밍그린", "크림레몬", "크림 그레이", "스페이스블랙", "실키크림", "샴페인골드", "실키오트밀",
              "크림그레이", "다크그레이", "모닝 블루", "그레이스화이트", "미스티 화이트", "에센셜 블루 그레이", "옥스포드그레이",
              "카밍클레이브라운", "사틴 차콜", "베르사유그레이", "매트블랙", "미스티라일락", "보타닉그린", "코튼플라워", "팬텀블랙", "시나모롤", "크림",
              "라이트블루", "다크레드", "라이트그린", "라임", "핑크골드", "딥퍼플", "화이트 티타늄", "블루 티타늄", "블랙 티타늄", "내추럴 티타늄",
              "실버", "베이지", "블랙", "화이트", "블루", "바이올렛", "옐로우", "그레이", "민트", "그린", "핑크", "퍼플", "차콜", "플래티넘", "옐로", "골드", "레드",
              "Black", "White", "Red", "Blue", "Green", "Yellow", "Pink", "Gray", "Silver", "Spacegray"]

general_stopwords_list = [ '에듀몰', '해피투게딜', '자급제', '재고보유', '하이마트배송!', 'WQXGA +', '30만원대',
                        '하이마트배송', '스태킹키트 포함', '스태킹키트 미포함', '한정판매', '한정수량', '온라인 전용', '1주이상소요/클리어런스', '스태킹/앵글',
                        '셀프관리형', '신모델', '즉시배송', '신상', '앵글설치포함,별도비용발생', '하이마트 설치', '최대36개월무', '사은품 증정', '제주마지막1대', '직렬설치',
                        '추가 다운로드 쿠폰', '국내정발', '빠른출발', '진열', '맞춤출수', '전시', '주연테크x하이메이드', '사내판매', '스태킹키트', '카드 추가',
                        '본품', '배송지연', '6개월주기 방문관리형', '익일배송', 'UP가전', '특가', '뭉치면싸다', '1만 다운로드 쿠폰', '1주차', '발송예정', '스태킹설치',
                        '사용안함', '스마트픽',  '미디어', '인버터', '정품OS', '단순배송', '2만원할인', '100원딜', '10대', '한정', '단독!', '단독', '추가 카드',
                        'AppleCare+', '1주이상소요', '**',  'DDR5', 'DDR4', 'DDR6', '시력보호', '최신형', '운영', '스태이션 포함!', '추가 쿠폰', '스태킹', '앵글',
                        '최종321만', '10대 한정판매',  '출고', '안드로이드', '배송시간 소요', '방문관리', 'i5.*?', 'i7.*?', 'win11', '20만원대', '링크!', '키트',
                        '사은품 증정', '2주이상 소요', '동시구매행사용', '1만원할인', '할인', '신학기특가', '키트포함', '키트 포함', 'CPU', 'GPU', 'RTX', 'GTX',
                        '각도조절형', '해피투게딜이벤트', '서랍형', '36개월케어십포함', '개별구매불가,본체만구매-자동취소', '전국무료설치', '8코어', '12코어', '16코어', '10코어',
                        '한정수량', '단종', '소진', '특가', '포토상품평이벤트', '어메이징','3월', '4월', '5월', '6월', '7월', '2주이상소요', '3주이상소요', 'NEW', '가벼운',
                        '8월', '9월', '10월', '11월', '12월', '1월', '2월', '행사', '1등급', '당질저감', '벽걸이조절형', '3년케어십', '초고온수', '기획전', '미포함', '포함',
                        '음성인식', '청정바람', '자가관리', '셀프형', '6개월', '방문주기', '시크릿 !', '최대', '10%', '신모델', '공동구매', '하이마트', '1만 다운로드 쿠폰',
                        '3개월', '창립기획', '단독모델', '스태킹설치포함', '자동먼지비움', '무빙키트' , '공기살균', '쇼핑몰', '상의5벌+하의1벌', '배송!', '외장', '외장그래픽',
                        '전자동', '오프라인', '온라인', '전용', '6인용', '3인용', '개별구매불가 본체만구매-자동취소', '미세먼지극복', '병렬설치', '비밀', '앵글설치포함']

## Exception functions

In [ ]:
# exceptions.py

def handle_process_text_exceptions(func):
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except AttributeError:
            print("MeCab object 'mecab' is not defined or doesn't have 'morphs' method")
            return "", ""
        except Exception as e:
            print(f"An error occurred in process_text: {str(e)}")
            return "", ""
    return wrapper

def handle_find_storage_exceptions(func):
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            print(f"An error occurred in find_storage: {str(e)}")
            return 'N/A', 'N/A'
    return wrapper

## Definition

In [ ]:
# @title
# Data cleaning Func
def cleaning_general(descriptions, serials, stopwords):
    cleaned_descriptions = []
    stopwords_set = set(stopwords)  # Convert the list to a set to remove duplicates

    for description, serial in zip(descriptions, serials):
        # Remove the serial number part
        serial_part = re.search(r'[A-Z]+-[A-Z0-9]+', serial)
        if serial_part:
            description = re.sub(rf'\[{re.escape(serial_part.group(0))}[^\]]*\]', '', description)

        # Continue with the original cleaning
        cleaned_description = re.sub(r'[^A-Za-z0-9ㄱ-ㅎㅏ-ㅣ가-힣+\s²]', '', description)
        for stopword in stopwords_set:
            cleaned_description = cleaned_description.replace(stopword, '')

        cleaned_descriptions.append(cleaned_description.strip())

    return cleaned_descriptions

# 제목 추출 함수
@handle_process_text_exceptions
def process_text(morphs, title_list):
    if not isinstance(text, str):
        raise TypeError("Input 'text' must be a string")
    if not isinstance(title_list, list):
        raise TypeError("Input 'title_list' must be a list")

    title = ''
    i = 0
    while i < len(morphs):
        if morphs[i] in title_list:
            title += morphs[i]
            i += 1
            while i < len(morphs) and morphs[i].isdigit():
                title += ' ' + morphs[i]
                i += 1
        else:
            i += 1

    return title.strip()

@handle_find_storage_exceptions
def find_storage(morphs):
    if not isinstance(morphs, list):
        raise TypeError("Input 'morphs' must be a list")
    storage = 'N/A'
    for word in morphs:
        if not isinstance(word, str):
            continue
        if word in ['64', '128', '256', '512', '1024']:
            storage = f'{word}GB'
        elif word in ['1TB', '2TB']:
            storage = word
    return storage

# Attribute call Func
def get_product_type(attributes, product_type_dict):
    for key in product_type_dict.keys():
        if key in attributes:
            return product_type_dict[key]
    return 'N/A'

# Duplicated words removal
def remove_duplicates(model_name, storage, ram, color):
    for word in [storage, color]:
        if word != 'N/A':
            model_name = model_name.replace(word, '').strip()

    return model_name

# Color Extract Func
def color_extraction(morphs,color_list):
  color = next((word for word in morphs if word in color_list), 'N/A')
  return color



In [ ]:
# Dataframe Creation
def create_dataframe(title, product_type_dict, serial, stopwords, color_list):
  filtered_text = cleaning_general(title, serial, stopwords)  # Data Cleaning
  extracted_title = pd.Series(dtype='object')
  extracted_color = pd.Series(dtype='object')
  extracted_storage = pd.Series(dtype='object')
  mecab = Mecab()
  for i in filtered_text:
    mecabmorphs = mecab.morphs(i) # 형태소 추출
    # 제목, 색상, 용량 형태소 추출
    extracted_title.append(process_text(mecabmorphs, product_title))
    extracted_color.append(color_extraction(mecabmorphs,color_list))
    extracted_storage.append(find_storage(mecabmorphs))
  cleaned_df = pd.DataFrame({'TITLE': extracted_title, 'COLOR': extracted_color, 'STORAGE': extracted_storage})
  return

# Add User Dictionary for Mecab

In [ ]:
cd /content/mecab-ko-dic-2.1.1-20180720

In [ ]:
!pip install jamo

In [ ]:
from jamo import h2j, j2hcj


def get_jongsung_TF(sample_text):
    sample_text_list = list(sample_text)
    last_word = sample_text_list[-1]
    last_word_jamo_list = list(j2hcj(h2j(last_word)))
    last_jamo = last_word_jamo_list[-1]

    jongsung_TF = "T"

    if last_jamo in ['ㅏ', 'ㅑ', 'ㅓ', 'ㅕ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ', 'ㅡ', 'ㅣ', 'ㅘ', 'ㅚ', 'ㅙ', 'ㅝ', 'ㅞ', 'ㅢ', 'ㅐ,ㅔ', 'ㅟ', 'ㅖ', 'ㅒ']:
        jongsung_TF = "F"

    return jongsung_TF

def make_user_dic_csv(morpheme_type, word_list, user_dic_file_name):
  file_data = []

  for word, score in word_list:
    jongsung_TF = get_jongsung_TF(word)

    line = f"{word},,,{score},{morpheme_type},*,{jongsung_TF},{word},*,*,*,*,*\n"
    # 단어, left-ID, right-ID, Weight, 품사, 의미분류, 종성유무, 읽기, 타입, 첫번째품사, 마지막품사, 표현
    # https://openuiz.blogspot.com/2018/12/mecab-ko-dic.html

    file_data.append(line)

  with open("./user-dic/user-nnp.csv", 'w', encoding='utf-8') as f:
    for line in file_data:
      f.write(line)

In [ ]:
word_list = [('1TB', 0), ('2TB', 0),  ('미패드',0), ('Wi-Fi',0)]

#  단어 추가 방법: ('원하는단어형태', 0) 를 대괄호 안에 넣어주세요. ( )엔 하나의 단어만 사용해주세요.
#  Method to add a word: Insert ('desired_word_form', 0) inside the bracket. Use only one word inside the parentheses.
#  Example) word_list = [('1TB', 0), ('2TB', 0)]

make_user_dic_csv(morpheme_type="NNP", word_list=word_list, user_dic_file_name='user-nnp.csv')

with open("./user-dic/user-nnp.csv", 'r', encoding='utf-8') as f:
  file_new = f.readlines()
file_new

In [ ]:
!bash autogen.sh

In [ ]:
!make

In [ ]:
!sudo make install

In [ ]:
!bash tools/add-userdic.sh

# Code Execution

In [ ]:
serial_num = df["ATTRIBUTES__MODEL_NUMBER__VALUE__KR"]
title = df["DESCRIPTION"]


# Debugging / Testing

In [ ]:
mecab = Mecab()

text = "   쿠쿠 17인용 대용량  CR-1755B 전기보온 밥솥  "
mecabnouns = mecab.nouns(text)
print("Mecab Nouns:", mecabnouns)
mecabmorphs = mecab.morphs(text)
print("Mecab Morphs:", mecabmorphs)
# cleaned_morphs = cleaning_general(mecabmorphs, general_stopwords_list,serial)
# print(cleaned_morphs)

# print(find_storage(cleaned_morphs))
# print(process_text(cleaned_morphs, product_title))
# print(color_extraction(cleaned_morphs,color_list))

Mecab Nouns: ['쿠쿠', '인용', '대용량', '전기', '보온', '밥솥']
Mecab Morphs: ['쿠쿠', '17', '인용', '대용량', 'CR', '-', '1755', 'B', '전기', '보온', '밥솥']
